In [1]:
# ✅ 1. 라이브러리 설치 및 인증
!pip install --upgrade gspread gspread_dataframe oauth2client scikit-learn

from google.colab import auth
auth.authenticate_user()

import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default
from collections import defaultdict

# ✅ 2. 구글 시트 연동
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1kkt336f1G-XqfDuwCUOnqpKlxTcnwLQy-XS4SQv6lM0/edit"
sh = gc.open_by_url(spreadsheet_url)
worksheet = sh.sheet1
data = worksheet.get_all_values()
headers = data[0]
rows = data[1:]
df = pd.DataFrame(rows, columns=headers)

# ✅ 3. 관심 분야 + 참가 목적 합치기
df["combined_text"] = df["관심 분야 (복수 응답 가능)"] + ", " + df["참가 목적 (복수 응답 가능)"]

# ✅ 4. 벡터화 및 K-Means 클러스터링
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["combined_text"])

k = 4  # 원하는 그룹 수 조정 가능
kmeans = KMeans(n_clusters=k, random_state=42)
df["자동분류그룹"] = kmeans.fit_predict(X)

# ✅ 5. 그룹 이름 부여 (예: 그룹 0, 그룹 1 ...)
df["자동분류그룹"] = df["자동분류그룹"].apply(lambda x: f"그룹 {x}")

# ✅ 6. 그룹당 최대 5명씩 소그룹 나누기
group_map = defaultdict(list)
for _, row in df.iterrows():
    group_map[row["자동분류그룹"]].append(row["이름"])

group_chunked_data = []
for group_name, members in group_map.items():
    for i in range(0, len(members), 5):
        chunk = members[i:i+5]
        numbered_group = f"{group_name} {i//5 + 1}조" if len(members) > 5 else group_name
        for m in chunk:
            others = [o for o in chunk if o != m]
            group_chunked_data.append([m, numbered_group, ", ".join(others)])

result_df = pd.DataFrame(group_chunked_data, columns=["참가자", "소속 그룹", "같은 그룹 참가자"])

# ✅ 7. 결과를 Sheet2에 저장
try:
    result_ws = sh.worksheet("Sheet2")
except:
    result_ws = sh.add_worksheet(title="Sheet2", rows="100", cols="20")

set_with_dataframe(result_ws, result_df)
print("✅ K-Means 분류 결과가 Sheet2에 저장되었습니다.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.1.4
    Uninstalling gspread-6.1.4:
      Successfully uninstalled gspread-6.1.4
✅ K-Means 분류 결과가 Sheet2에 저장되었습니다.
